# Testing

A notebook to test the implementation

In [1]:
import gzip
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## 0 Preprocessing

For testing, we use the `MNIST` data. We first extract the data from `gzip` file.

In [2]:
# prepare data
def get_imgs(dir):
    with gzip.open(dir, 'r') as f:
        # first 4 bytes (some number)
        _ = int.from_bytes(f.read(4), 'big')
        # no. of images
        num_imgs = int.from_bytes(f.read(4), 'big')
        # row count
        row_cnt = int.from_bytes(f.read(4), 'big')
        # column count
        col_cnt = int.from_bytes(f.read(4), 'big')

        img_data = f.read()
        images = np.frombuffer(img_data, dtype=np.uint8).\
            reshape((num_imgs, row_cnt, col_cnt))
        return images

def get_labels(dir):
    with gzip.open(dir, 'r') as f:
        _ = int.from_bytes(f.read(4), 'big')
        label_cnt = int.from_bytes(f.read(4), 'big')
        print(label_cnt)
        label_data = f.read()
        labels = np.frombuffer(label_data, dtype=np.uint8)
        return labels

We now make `dataset`. Note that we used the `tf.data.Dataset.from_tensor_slices` to create dataset from a tuple of numpy arrays; if they were instead lists, then there will be an error

In [3]:
img_train    = get_imgs('data/train-images-idx3-ubyte.gz')
label_train  = get_labels('data/train-labels-idx1-ubyte.gz')
ds_train     = tf.data.Dataset.from_tensor_slices((img_train, label_train))

img_test     = get_imgs('data/t10k-images-idx3-ubyte.gz')
label_test   = get_labels('data/t10k-labels-idx1-ubyte.gz')
ds_test      = tf.data.Dataset.from_tensor_slices((img_test, label_test))

60000
10000


Check by visualizing. Uncomment the last line to visualize.

In [4]:
def look(ds):
    for img,label in ds:
        plt.imshow(img.numpy().astype('uint8'))
        print(label)
        print(np.linalg.norm(img))
        break
# look(ds_train)

Preprocess the dataset: normalizing, shuffling, and batching

In [5]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return (tf.cast(image, tf.float32)/255.0 , label)  

def preProcess(dset:tf.data.Dataset, batch_size = 128):
  '''Pre-processes the dataset.'''
  dset = dset.map(normalize_img)
  dset = dset.shuffle(len(dset))
  dset = dset.batch(batch_size)
  return dset

In [6]:
ds_train = preProcess(ds_train)
ds_test = preProcess(ds_test)

In [7]:
def reshapeImg(img, label):
    return (tf.reshape(img, (-1,784)), label)
    
# for img,label in ds_train.map(reshapeImg):
#     # plt.imshow(img[0])
#     print(label[0])
#     print(np.linalg.norm(img[0]))
#     print(img.shape)
#     break

## 1 Running experiment

First define some hyperparameters

In [8]:
layers = [784, 10] # use [784, 100, 10]

activation = 'relu'

lr = 0.01

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

metrics = tf.keras.metrics.SparseCategoricalAccuracy()    

train_loss = tf.keras.metrics.Mean

train_acc = tf.keras.metrics.SparseCategoricalAccuracy  

epochs = 1  

num_pruning = 2  

step_perc = 0.5

In [9]:
from pruning import *
from models import *

model_params = {'layers': layers,
                'activation': 'relu',
                'BatchNorm':False,
                'Dropout':None,
                'optimizer': tf.keras.optimizers.Adam(lr),
                'loss_fn': loss_fn,
                'metrics': metrics
                }

train_params = {'train_loss': train_loss,
                'train_acc': train_acc
                }

prune_exp = pruning(ds_train.map(reshapeImg), ds_test.map(reshapeImg), model_params,
                    train_params,
                    epochs, num_pruning, step_perc)

prune_exp.makeModel()
# prune_exp.test_run()

Model: "ModeltoPrune"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 784)               615440    
                                                                 
 dense_1 (Dense)             (None, 10)                7850      
                                                                 
Total params: 623,290
Trainable params: 623,290
Non-trainable params: 0
_________________________________________________________________


In [10]:
prune_exp.prune()


 
 Iterative pruning round: 0 
 

Model: "ModeltoPrune"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 784)               615440    
                                                                 
 dense_3 (Dense)             (None, 10)                7850      
                                                                 
Total params: 623,290
Trainable params: 623,290
Non-trainable params: 0
_________________________________________________________________

 Start original model training. 

Epoch 0 for original 

Model: "ModeltoPrune"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 784)               615440    
                                                                 
 dense_5 (Dense)             (None, 10)                7850      